In [16]:
import json
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches
import numpy as np

In [6]:
import json
import os
import numpy as np

def convert_coco_to_yolo(coco_annotations_path, images_dir, output_dir):
    with open(coco_annotations_path, 'r') as f:
        data = json.load(f)

    images = {image['id']: image for image in data['images']}
    annotations = data['annotations']
    categories = {category['id']: category for category in data['categories']}

    category_name_to_id = {category['name']: idx for idx, category in enumerate(categories.values())}

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)

    for annotation in annotations:
        image_id = annotation['image_id']
        image_info = images[image_id]
        image_filename = image_info['file_name']
        image_width = image_info['width']
        image_height = image_info['height']
        image_path = os.path.join(images_dir, image_filename)

        if not os.path.exists(image_path):
            continue

        output_image_path = os.path.join(output_dir, 'images', image_filename)
        output_label_path = os.path.join(output_dir, 'labels', os.path.splitext(image_filename)[0] + '.txt')

        os.system(f'cp "{image_path}" "{output_image_path}"')

        with open(output_label_path, 'a') as f:
            category_name = categories[annotation['category_id']]['name']
            category_id = category_name_to_id[category_name]
            
            # Normalize bounding box coordinates
            bbox = annotation['bbox']
            bbox[0] /= image_width  # x
            bbox[1] /= image_height # y
            bbox[2] /= image_width  # width
            bbox[3] /= image_height # height
            bbox_str = ' '.join(map(str, bbox))

            # Normalize segmentation points
            segmentation = annotation['segmentation'][0]
            segmentation = np.array(segmentation).reshape((-1, 2))
            segmentation[:, 0] /= image_width   # x points
            segmentation[:, 1] /= image_height  # y points
            segmentation_str = ' '.join(map(str, segmentation.flatten()))

            f.write(f"{category_id} {bbox_str} {segmentation_str}\n")


In [7]:
# Paths to your data
coco_annotations_path = 'SAF22_COCO_CorentinDoche.json'
images_dir = 'images'  # Change this to your image directory
output_dir = 'output'  # Directory where YOLO formatted data will be saved

convert_coco_to_yolo(coco_annotations_path, images_dir, output_dir)

In [11]:
from ultralytics import YOLO

In [12]:
model = YOLO("yolov8n-seg.pt")  # load a pretrained model (recommended for training)

In [17]:
# Train the model
results = model.train(data="C:/Users/devan/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml", epochs=5, imgsz=640)


Ultralytics YOLOv8.2.61  Python-3.12.4 torch-2.3.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
print(results.class_result(0))
print(results.class_result(1))
print(results.class_result(2))
print(results.class_result(3))